In [4]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage
import cv2
import glob
import os


class call_num():
    def __init__(self):
        self.number = 0
    def add(self):
        self.number = self.number + 1
    def print(self):
        return self.number

def translateit(image, position_x, position_y, roi_size):
    return image[position_y:position_y+roi_size, position_x:position_x+roi_size]

def rotateit(image, theta):
    return scipy.ndimage.rotate(image, float(theta), reshape=False, mode='nearest')

def flipit(image, bool, type = 'ver'):
    if bool == 1:
        if type == 'ver':
            image = np.fliplr(image)
        else:
            image = np.flip(image)
    else:
        image = image
    return image

def augmentation_data(image, roi_size=512):
    w, h = np.shape(image)[1], np.shape(image)[0]

    random_angle = np.random.choice([0, 90, 180, 270])
    random_flip_ho = np.random.randint(0,2)
    random_flip_ver = np.random.randint(0,2)
    random_y = np.random.randint(0,h-roi_size)
    random_x = np.random.randint(0,w-roi_size)
    # random_flip_ver = np.random.randint(0,2)

    image= np.expand_dims(image,axis=2)
    image = translateit(image, random_x, random_y, roi_size)
    image = flipit(image,random_flip_ho, type='ho')
    image = flipit(image,random_flip_ver, type='ver')
    image = rotateit(image,random_angle)
    return image

def test_data(image, roi_size=512):
    w, h = np.shape(image)[1], np.shape(image)[0]
    random_y = np.random.randint(0,h-roi_size)
    random_x = np.random.randint(0,w-roi_size)

    image= np.expand_dims(image,axis=2)
    image = translateit(image, random_x, random_y, roi_size)

    return image

In [5]:
import glob
import pandas as pd
import tqdm
idx = call_num()

aug_num = 30
df = pd.read_excel('../data/raw_data.xlsx')

for concentration,deposition,spin_cationg_speed,sheet_resistance,transmittance in np.array(df):
    path = '../data/raw_data/{}/{}/'.format(str(concentration), str(int(spin_cationg_speed)))
    for image_path in tqdm.tqdm(glob.glob(path+'*jpg')):
            for num in range(aug_num):
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)[0:890,:]
                aug_data = augmentation_data(image, 512)
                np.save('../data/train_set/{}_{}_{}'.format(str(idx.print()).zfill(4),str(num), str(sheet_resistance)), aug_data)
                cv2.imwrite('../data/train_check/{}_{}_{}.jpg'.format(str(idx.print()).zfill(4),str(+num), str(sheet_resistance)), aug_data)
                idx.add()




100%|██████████| 3/3 [00:04<00:00,  1.61s/it]


In [5]:
import glob
import pandas as pd

idx = call_num()

df = pd.read_excel('../data/raw_data.xlsx')
num = 0
for concentration,deposition,spin_cationg_speed,sheet_resistance,transmittance in np.array(df):
    path = '../data/raw_data/{}/{}/'.format(str(concentration), str(int(spin_cationg_speed)))
    for image_path in glob.glob(path+'*jpg'):
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)[0:890,:]
        aug_data = test_data(image, 512)
        np.save('../data/test_set/{}_{}_{}'.format(str(idx.print()).zfill(4),str(num), str(sheet_resistance)), aug_data)
        cv2.imwrite('../data/test_check/{}_{}_{}.jpg'.format(str(idx.print()).zfill(4),str(+num), str(sheet_resistance)), aug_data)
        idx.add()